<a href="https://colab.research.google.com/github/Jakubelo/SystemRecomendation/blob/main/SystemRecomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! ls "drive/MyDrive/Colab Notebooks/NLP/data/archive"

Combined_Jobs_Final.csv  job_data.csv	Positions_Of_Interest.csv
Experience.csv		 Job_Views.csv


Creating Jobs Corpus

In [3]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/Colab Notebooks/NLP/data/archive/Combined_Jobs_Final.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84090 entries, 0 to 84089
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Job.ID              84090 non-null  int64  
 1   Provider            84090 non-null  int64  
 2   Status              84090 non-null  object 
 3   Slug                84090 non-null  object 
 4   Title               84090 non-null  object 
 5   Position            84090 non-null  object 
 6   Company             81819 non-null  object 
 7   City                83955 non-null  object 
 8   State.Name          83919 non-null  object 
 9   State.Code          83919 non-null  object 
 10  Address             36 non-null     object 
 11  Latitude            84090 non-null  float64
 12  Longitude           84090 non-null  float64
 13  Industry            267 non-null    object 
 14  Job.Description     84034 non-null  object 
 15  Requirements        0 non-null      float64
 16  Sala

In [4]:
df = df[['Job.ID', 'Title', 'Position', 'Company', 'City', 'Employment.Type', 'Job.Description']]
df.columns = ['Job.ID', 'Title', 'Position', 'Company','City', 'Empl_type','Job_Description']

In [5]:
df["text"] = df["Position"].map(str) + " " + df["Company"] +" "+ df["City"]+ " "+df['Empl_type']+" "+df['Job_Description'] +" "+df['Title']

In [6]:
df_all = df[['Job.ID', 'text', 'Title']]

In [7]:
df_all = df_all.fillna(" ")

In [8]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
import re
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   

def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "",text)
  text=re.sub("(\\d|\\W)+"," ",text) 
  text = text.replace("nbsp", "")
  clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
  clean_text2 = [word for word in clean_text if black_txt(word)]
  return " ".join(clean_text2)

In [10]:
df_all['text'] = df_all['text'].apply(clean_txt)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_jobid = tfidf_vectorizer.fit_transform((df_all['text'])) #fitting and transforming the vector
tfidf_jobid

<84090x50767 sparse matrix of type '<class 'numpy.float64'>'
	with 8249764 stored elements in Compressed Sparse Row format>

# Creating User Corpus

Job Views Data

In [14]:
df_job_view = pd.read_csv("drive/MyDrive/Colab Notebooks/NLP/data/archive/Job_Views.csv")
df_job_view.head(2)

,Applicant.ID,Job.ID,Title,Position,Company,City,State.Name,State.Code,Industry,View.Start,View.End,View.Duration,Created.At,Updated.At
0,10000,73666,Cashiers & Valets Needed! @ WallyPark,Cashiers & Valets Needed!,WallyPark,Newark,New Jersey,NJ,NaN,2014-12-12 20:12:35 UTC,2014-12-12 20:31:24 UTC,1129.0,2014-12-12 20:12:35 UTC,2014-12-12 20:12:35 UTC
1,10000,96655,Macy's Seasonal Retail Fragrance Cashier - Ga...,Macy's Seasonal Retail Fragrance Cashier - Ga...,Macy's,Garden City,New York,NY,NaN,2014-12-12 20:08:50 UTC,2014-12-12 20:10:15 UTC,84.0,2014-12-12 20:08:50 UTC,2014-12-12 20:08:50 UTC


In [17]:
df_job_view = df_job_view[['Applicant.ID', 'Job.ID', 'Position', 'Company','City']]
df_job_view["select_pos_com_city"] = df_job_view["Position"].map(str) + "  " + df_job_view["Company"] +"  "+ df_job_view["City"]
df_job_view['select_pos_com_city'] = df_job_view['select_pos_com_city'].map(str).apply(clean_txt)
df_job_view['select_pos_com_city'] = df_job_view['select_pos_com_city'].str.lower()
df_job_view = df_job_view[['Applicant.ID','select_pos_com_city']]
df_job_view.head()

,Applicant.ID,select_pos_com_city
0,10000,cashier valet need wallypark newark
1,10000,macys seasonal retail fragrance cashier garden...
2,10001,part time showroom sales cashier grizzly indus...
3,10002,event specialist part time advantage sales mar...
4,10002,bonefish kitchen staff bonefish grill greenville


Experience Data

In [19]:
df_experience = pd.read_csv("drive/MyDrive/Colab Notebooks/NLP/data/archive/Experience.csv")
df_experience.head(2)

,Applicant.ID,Position.Name,Employer.Name,City,State.Name,State.Code,Start.Date,End.Date,Job.Description,Salary,Can.Contact.Employer,Created.At,Updated.At
0,10001,Account Manager / Sales Administration / Quali...,Barcode Resourcing,Bellingham,Washington,WA,2012-10-15,NaN,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC
1,10001,Electronics Technician / Item Master Controller,Ryzex Group,Bellingham,Washington,WA,2001-12-01,2012-04-01,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC


In [21]:
df_experience = df_experience[['Applicant.ID', 'Position.Name']]
df_experience['Position.Name'] = df_experience['Position.Name'].map(str).apply(clean_txt)
df_experience.head(3)

,Applicant.ID,Position.Name
0,10001,account manager sales administration quality a...
1,10001,electronics technician item master controller
2,10001,machine operator


In [24]:
df_experience =  df_experience.sort_values(by='Applicant.ID')
df_experience = df_experience.fillna(" ")
df_experience.head()

,Applicant.ID,Position.Name
2763,2,volunteer
2762,2,writer uloop blog
3759,3,market intern
3758,3,server
3757,3,prep cook


In [29]:
df_experience = df_experience.groupby(by='Applicant.ID')['Position.Name'].apply(' '.join).reset_index()
df_experience.head(3)

,Applicant.ID,Position.Name
0,2,volunteer writer uloop blog
1,3,market intern server prep cook
2,6,project assistant


Position of interest Data

In [31]:
df_poi =  pd.read_csv("drive/MyDrive/Colab Notebooks/NLP/data/archive/Positions_Of_Interest.csv")
df_poi.head(3)

,Applicant.ID,Position.Of.Interest,Created.At,Updated.At
0,10003,security officer,2014-12-12 21:20:54 UTC,2014-12-12 21:20:54 UTC
1,10007,Server,2014-08-14 15:56:42 UTC,2015-02-26 20:35:12 UTC
2,10007,Bartender,2014-08-14 15:56:44 UTC,2015-02-19 23:21:28 UTC


In [33]:
df_poi = df_poi[['Applicant.ID', 'Position.Of.Interest']]
df_poi['Position.Of.Interest']=df_poi['Position.Of.Interest'].map(str).apply(clean_txt)
df_poi = df_poi.fillna(" ")
df_poi.head(3)

,Applicant.ID,Position.Of.Interest
0,10003,security officer
1,10007,server
2,10007,bartender


In [34]:
df_poi = df_poi.groupby('Applicant.ID', sort=True)['Position.Of.Interest'].apply(' '.join).reset_index()
df_poi.head(3)

,Applicant.ID,Position.Of.Interest
0,96,server
1,153,server host barista customer service rep sales...
2,256,server host receptionist book keeper customer ...


In [ ]:
df_

In [35]:
df_jobs_exp = df_job_view.merge(df_experience, left_on='Applicant.ID', right_on='Applicant.ID', how='outer')
df_jobs_exp = df_jobs_exp.fillna(' ')
df_jobs_exp = df_jobs_exp.sort_values(by='Applicant.ID')
df_jobs_exp.head()

,Applicant.ID,select_pos_com_city,Position.Name
12370,2,,volunteer writer uloop blog
12371,3,,market intern server prep cook
12372,6,,project assistant
12373,8,,deli clerk server cashier food prep order taker
12374,11,,cashier


In [48]:
df_jobs_exp_poi = df_jobs_exp.merge(df_poi, left_on='Applicant.ID', right_on='Applicant.ID', how='outer')
df_jobs_exp_poi = df_jobs_exp_poi.fillna(' ')

In [49]:
df_jobs_exp_poi.head(3)

,Applicant.ID,select_pos_com_city,Position.Name,Position.Of.Interest
0,2,,volunteer writer uloop blog,
1,3,,market intern server prep cook,
2,6,,project assistant,


In [60]:
df_jobs_exp_poi["text"] = df_jobs_exp_poi["select_pos_com_city"].map(str) + df_jobs_exp_poi["Position.Name"] +" "+ df_jobs_exp_poi["Position.Of.Interest"]
df_person_profile = df_jobs_exp_poi[['Applicant.ID', 'text']]
df_person_profile.columns = ['applicant_id', 'text']
df_person_profile['text'] = df_person_profile['text'].apply(clean_txt)
df_person_profile.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,applicant_id,text
0,2,volunteer writer uloop blog
1,3,market intern server prep cook
2,6,project assistant
3,8,deli clerk server cashier food prep order taker
4,11,cashier


In [61]:
df_person_profile.head(3)

,applicant_id,text
0,2,volunteer writer uloop blog
1,3,market intern server prep cook
2,6,project assistant


In [62]:
df_person_profile.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15959 entries, 0 to 15958
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   applicant_id  15959 non-null  int64 
 1   text          15959 non-null  object
dtypes: int64(1), object(1)
memory usage: 374.0+ KB


In [92]:
user_id = 111
user_q = df_person_profile[df_person_profile.applicant_id == user_id]
user_q

,applicant_id,text
64,111,cashier


# The Recomendation System

In [71]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

count_jobid = count_vectorizer.fit_transform((df_all['text'])) #fitting and transforming the vector
count_jobid

<84090x50767 sparse matrix of type '<class 'numpy.int64'>'
	with 8249764 stored elements in Compressed Sparse Row format>

In [74]:
from sklearn.metrics.pairwise import cosine_similarity
user_count = count_vectorizer.transform(user_q['text'])
cos_similarity_countv = map(lambda x: cosine_similarity(user_count, x),count_jobid)

In [75]:
output2 = list(cos_similarity_countv)

In [82]:
len(output2)

84090

In [83]:
df_all

,Job.ID,text,Title
0,111,server tacolicious palo alto part time tacolic...,Server @ Tacolicious
1,113,kitchen staff chef claude lane san francisco p...,Kitchen Staff/Chef @ Claude Lane
2,117,bartender machka restaurants corp san francisc...,Bartender @ Machka Restaurants Corp.
3,121,server teriyaki house brisbane part time serve...,Server @ Teriyaki House
4,127,kitchen staff chef rosa mexicano sunset los an...,Kitchen Staff/Chef @ Rosa Mexicano - Sunset
...,...,...,...
84085,82,book keeper national japanese american histori...,Book Keeper @ National Japanese American Histo...
84086,83,kitchen staff chef emporio rulli larkspur part...,Kitchen Staff/Chef @ Emporio Rulli
84087,84,driver onigilly san francisco part time onigil...,Driver @ Onigilly
84088,88,line cook machka restaurants corp san francisc...,Line Cook @ Machka Restaurants Corp.


In [86]:
def get_recommendation(top, df_all, scores, userid):
  recommendation = pd.DataFrame(columns = ['ApplicantID', 'JobID',  'title', 'score'])
  count = 0
  for i in top:
      recommendation.at[count, 'ApplicantID'] = userid
      recommendation.at[count, 'JobID'] = df_all['Job.ID'][i]
      recommendation.at[count, 'title'] = df_all['Title'][i]
      recommendation.at[count, 'score'] =  scores[count]
      count += 1
      
  return recommendation

In [91]:
user_q

,applicant_id,text
64,111,cashier


In [93]:
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:10]
list_scores = [output2[i][0][0] for i in top]
get_recommendation(top, df_all, list_scores, user_id) 

,ApplicantID,JobID,title,score
0,111,92,Cashier @ Kazoo Restaurant,0.588348
1,111,151,Cashier @ Kissako Tea,0.53936
2,111,270378,Cashier @ Northern Virginia Community College,0.512148
3,111,306666,Cashiers @ Helpmates Staffing Services.,0.496139
4,111,146514,Billing Clerk @ Accountemps,0.481125
5,111,265170,Cashier @ Sears Hometown and Outlet,0.464991
6,111,315594,Cashier @ OfficeTeam,0.441726
7,111,308431,General Cashier (Part Time) @ Parallon,0.440225
8,111,259605,Service Cashier @ Chapman Auto Group,0.437595
9,111,418,Cashier @ Santek Shop,0.424264
